# groupby and finding the right category/bins/places to slice them in groupbys

In [1]:
import pandas as pd
import sqlite3
cnx = sqlite3.connect(':memory:')

In [2]:
csvfile = ('/Users/randy/Documents/GitHub/Pokemon-Stat-Predictor/Pokemon.csv') #Original data

In [3]:
columns = ['#','name','type1','type2','total','hp','attack','defense',\
           'sp_atk','sp_def','speed','generation','legendary']
#open the csv file
df = pd.read_csv(csvfile, names=columns, header=0)

In [4]:
#find NaN values
nan_rows = df[df.isnull().T.any().T]
nan_rows.head()

,#,name,type1,type2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False
10,8,Wartortle,Water,NaN,405,59,63,80,65,80,58,1,False
11,9,Blastoise,Water,NaN,530,79,83,100,85,105,78,1,False


In [5]:
#change all Type 2 NaN values to 'None':
df['type2'] = df['type2'].fillna('none')

In [6]:
#check for NaN values again
nan_rows = df[df.isnull().T.any().T]
nan_rows.head()

,#,name,type1,type2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary


In [7]:
#change all strings within the dataframe to lower case
df = df.astype(str).apply(lambda x: x.str.lower())

In [8]:
df.head()

,#,name,type1,type2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,1,bulbasaur,grass,poison,318,45,49,49,65,65,45,1,false
1,2,ivysaur,grass,poison,405,60,62,63,80,80,60,1,false
2,3,venusaur,grass,poison,525,80,82,83,100,100,80,1,false
3,3,venusaurmega venusaur,grass,poison,625,80,100,123,122,120,80,1,false
4,4,charmander,fire,none,309,39,52,43,60,50,65,1,false


In [9]:
#set the database for pokemon
df.to_sql('pokemon', con=cnx, if_exists='append', index=False)

In [10]:
#function for the SQL queries below
def sql_query(query):
    return pd.read_sql(query, cnx)

# Some Group By examples

In [26]:
#simple MAX
query = '''
SELECT name, type1, type2, MAX(total)
FROM pokemon
WHERE legendary = 'true';
'''

sql_query(query)

,name,type1,type2,MAX(total)
0,mewtwomega mewtwo x,psychic,fighting,780


In [27]:
#MAX GROUP BY type1
query = '''
SELECT name, type1, type2, MAX(total)
FROM pokemon
WHERE legendary = 'true'
GROUP BY type1;
'''

sql_query(query)

,name,type1,type2,MAX(total)
0,yveltal,dark,flying,680
1,rayquazamega rayquaza,dragon,flying,780
2,zapdos,electric,flying,580
3,xerneas,fairy,none,680
4,ho-oh,fire,flying,680
5,tornadusincarnate forme,flying,none,580
6,giratinaaltered forme,ghost,dragon,680
7,shayminland forme,grass,none,600
8,groudonprimal groudon,ground,fire,770
9,articuno,ice,flying,580


# HAVING Clause

In [45]:
#total HP group by type1, SUM(HP) > 4000

query = '''
SELECT COUNT(name) as pokemon_count, type1, MIN(total), MAX(total), AVG(HP)
FROM pokemon
GROUP BY type1
HAVING SUM(HP) > 4000;
'''

sql_query(query)

,pokemon_count,type1,MIN(total),MAX(total),AVG(HP)
0,70,grass,180,630,67.271429
1,98,normal,190,720,77.275510
2,57,psychic,198,780,70.631579
3,112,water,200,770,72.062500


# Subqueries in the SELECT, FROM, WHERE statements

# GROUP BY should only be used with the aggregate functions (COUNT, SUM, MAX, MIN, AVG)

In [12]:
query = '''
SELECT type1, type2, MIN(total), MAX(total)
FROM pokemon
GROUP BY type1, type2;
'''

sql_query(query)

,type1,type2,MIN(total),MAX(total)
0,bug,electric,319,472
1,bug,fighting,500,600
2,bug,fire,360,550
3,bug,flying,244,600
4,bug,ghost,236,236
5,bug,grass,285,500
6,bug,ground,266,424
7,bug,none,194,500
8,bug,poison,195,495
9,bug,rock,325,505


In [13]:
query = '''
SELECT type1, COUNT(total), SUM(total), MIN(total), MAX(total)
FROM pokemon
WHERE total > 100
GROUP BY type1;
'''

sql_query(query)

,type1,COUNT(total),SUM(total),MIN(total),MAX(total)
0,bug,69,26146,194,600
1,dark,31,13818,220,680
2,dragon,32,17617,300,780
3,electric,44,19510,205,610
4,fairy,17,7024,218,680
5,fighting,27,11244,210,625
6,fire,52,23820,250,680
7,flying,4,1940,245,580
8,ghost,32,14066,275,680
9,grass,70,29480,180,630


In [14]:
df.head()

,#,name,type1,type2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,1,bulbasaur,grass,poison,318,45,49,49,65,65,45,1,false
1,2,ivysaur,grass,poison,405,60,62,63,80,80,60,1,false
2,3,venusaur,grass,poison,525,80,82,83,100,100,80,1,false
3,3,venusaurmega venusaur,grass,poison,625,80,100,123,122,120,80,1,false
4,4,charmander,fire,none,309,39,52,43,60,50,65,1,false
